In [56]:
from Bio import SeqIO
from Bio import SeqFeature
from Bio.Alphabet import IUPAC
import pandas as pd
from Levenshtein import distance
import numpy as np
import csv
from ipywidgets import IntProgress
from IPython.display import display
import multiprocessing as mp
import time
from tqdm import tqdm_notebook as tqdm
from itertools import product, combinations, permutations
from os import listdir, mkdir
import pickle
import timeit
from iterated_tasks import Task
import edlib
import array
from collections import Counter
from pickle import UnpicklingError
#ocf cluster

ModuleNotFoundError: No module named 'Levenshtein'

In [42]:
genomes = [f for f in listdir("data") if "_" not in f]
# genomes.remove("Humans")
alu = ["AluJb", "AluJo", "AluJr", "AluJr4", "AluSc", "AluSc5", "AluSc8", "AluSg", "AluSg4", "AluSg7", "AluSp", "AluSq", "AluSq10", "AluSq2", "AluSq4", "AluSx", "AluSx1", "AluSx3", "AluSx4", "AluSz", "AluSz6", "AluY"]
len(list(product(permutations(genomes, 2), alu)))

tasks = list(product(product(["Humans"], genomes[:7][::-1]), alu))
create_tasks(tasks)

3432

In [9]:
errors = []
for f in listdir("tasks"):
    try:
        task = pickle.load(file=open("tasks/" + f, "rb"))
    except UnpicklingError:
        errors.append(f)
#     task.subsection = None
#     pickle.dump(task, open("tasks/"+task.filename(), "wb"))
# pickle.dump(t, open("tasks/"+t.filename(), "wb"))

file = errors[0]
names = file.split("_")
task = Task(((names[0], names[1]), names[2].split(".")[0]))
with open("results/"+file+".csv") as f:
    task.completed = sum(1 for line in f) - 

In [119]:
c = Counter([1, 1, 1, 1, 1, 2, 3, 4, 3, 4, 3, 4, 4, 4, 4, 1, 1])
def get_order(c):
    ordering = {}
    for key, value in c.most_common():
        ordering[key] = len(ordering)
    def order(chromosome):
        if chromosome in ordering.keys():
            return ordering[chromosome]
        else:
            return len(ordering)
    return order
get_order(c)(7)

4

In [16]:
for file in errors:
    names = file.split("_")
    task = Task(((names[0], names[1]), names[2].split(".")[0]))
    with open("results/"+file+".csv") as f:
        task.completed = sum(1 for line in f) - 1
    pickle.dump(task, open("tasks/"+file, "wb"))
    

In [167]:
task = Task((("Humans", "Chimps"), "AluY"))
task.completed = 100000
batch_size = 100
species1_records = pickle.load(open("data/" + task.species1 + "/" + task.species1 + "_" + task.subfamily + ".p", "rb"))[task.completed:task.completed+batch_size]
species2_records = pickle.load(open("data/" + task.species2 + "/" + task.species2 + "_" + task.subfamily + ".p", "rb"))
species2_chr = [get_location(sequence2.description)[0] for sequence2 in species2_records]

In [169]:
t = time.time()
times = []
datas = []
c = Counter()
for sequence1 in species1_records:
    order = get_order(c)
#     [sequence2 for _, x in sorted(zip(species2_chr,species2_records), key=lambda pair: order(pair[0]))]
    # matches = [editdistance.eval(str(sequence1.seq), str(sequence2.seq)) for sequence2 in species2_records]
    k = 300
    match = ""
    new_order = sorted(zip(species2_chr,species2_records), key=lambda pair: order(pair[0]))
    t1 = time.time()
    for _, sequence2 in new_order:
#     for sequence2 in species2_records:
        k_new = edlib.align(str(sequence1.seq), str(sequence2.seq), task = "editDistance", k = k)["editDistance"]
        if k_new != -1 and k_new < k:
            match = sequence2
            k = k_new
    times.append(time.time()-t1)
    # min_index = np.argmin(matches)
    # match = species2_records[min_index]
    location1 = get_location(sequence1.description)
    location2 = get_location(match.description)
    c[location2[0]] += 1
    data = np.concatenate([location1, location2, [k], [abs(location1[1] - location2[1])]])
    datas.append(data)
# print(datas[0])
# print(datas[-1])
print(np.average(times))
time.time() - t

0.6553196334838867


82.04370713233948

In [162]:
t = time.time()
# sorted(species2_chr)
[x for _, x in sorted(zip(species2_chr,species2_records), key=lambda pair: order(pair[0]))][0]
time.time() - t

0.052996158599853516

In [3]:
tasks = [pickle.load(file=open("tasks/" + f, "rb")) for f in listdir("tasks")]
# started = [task for task in tasks if task.species1 == "Humans"]
finished = [task for task in tasks if task.finished()]
print(len(finished))
# df = pd.read_csv("results/"+started[0].filename()+".csv", header=None)

AttributeError: Can't get attribute 'Task' on <module '__main__'>

In [59]:
genomes = [f for f in listdir("data") if "_" not in f]
genomes.remove("Humans")
for a in alu[:1]:
    for genome in genomes[:1]:
        string = []
        for task in started:
            if task.species2 == genome and task.subfamily == alu:
                df = pd.read_csv("results/"+task.filename()+".csv", header=None)
                print(df['8'].str.cat)

In [56]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,chr1,192937858,192938175,chr1,171366085,171366402,24,21571773,GCCGGGCATGGTGGCTCACTCTTGTTAATCCCAGCACTTTGGGAAG...,GCCGGGCATGGTGGCTCATTCTTGTTAATCCCAGCACTTTGGGAAG...
1,chr1,24117116,24117383,chr1,26355737,26356001,25,2238621,CCCAGCACTTTGGGAGGCCAAGGTGGAGGATCACTTGGGTCCAGGA...,CCCAGCACTTTGGGAGGCCAAGGCGGAGGATCACTTGAGCCCAGGA...
2,chr1,32505606,32505884,chr1,34701911,34702187,25,2196305,GGCCAGGTGGTGGCTTATATCTGTAATCCCACCACTTTGGGAGGCT...,GGCCAGGTGGTGGCTTATATCTATAATCCCATCACTTTGGGAGGCT...
3,chr1,54525648,54525956,chr9,95122753,95123061,48,40597105,GGCCAGGCACAGCGGCTCACACCTGTAATCCCAGCACTTTGGGAGG...,GGCCAGGTGCAGGGGCTCACGCCTATAATCCTAGTACTTTGGGAGG...
4,chr1,236977955,236978230,chr1,212128917,212129192,26,24849038,GGCTGGGCGTGGTGGCTCATGCCTATAATCCCAGCACTTTGGGAAG...,GGCTGGGTGTGGTGGCTCATGCCTATAATCCCAGCACTTTGGGAAG...


In [7]:
genomes = [f.split(".")[0] for f in listdir("data") if ".fasta" in f]
alu = ["AluJb", "AluJo", "AluJr", "AluJr4", "AluSc", "AluSc5", "AluSc8", "AluSg", "AluSg4", "AluSg7", "AluSp", "AluSq", "AluSq10", "AluSq2", "AluSq4", "AluSx", "AluSx1", "AluSx3", "AluSx4", "AluSz", "AluSz6", "AluY"]
create_pickles(genomes)

In [118]:
t = Task(tasks[0])

In [119]:
pickle.dump(obj=t, file=open(t.filename(), "wb"))

In [120]:
t = pickle.load(file=open(t.filename(), "rb"))

In [50]:
#get the frequencies for each subfamily in a genome
#ex: get_frequencies("humans")
class Task():
    def __init__(self,task):
        self.species1 = task[0][0]
        self.species2 = task[0][1]
        self.subfamily = task[1]
        self.total = len(pickle.load(file=open("data/"+self.species1 + "/" + self.species1+"_"+self.subfamily + ".p", "rb")))
        self.completed = 0
    def filename(self):
        return self.species1 + "_" + self.species2 + "_" + self.subfamily + ".p"
    def update(self,amount):
        self.completed += amount
    def finished(self):
        return self.completed == self.total
    def remaining(self):
        return self.total - self.completed
    def __eq__(self, obj):
        return (self.species1 == obj.species1) & (self.species2 == obj.species2) & (self.subfamily == obj.subfamily)
def create_tasks(tasks):
#warning: only run once, will reset progress for tasks
    for task in tasks:
        t = Task(task)
        pickle.dump(t, open("tasks/"+t.filename(), "wb"))
#     task_list = [Task(task) for task in tasks]
#     pickle.dump(obj=task_list, file=open("progress.p", "wb"))
def create_pickles(genomes):
    d = {}
    for g in genomes:
        records = SeqIO.parse("data/" + g + ".fasta", "fasta")
        for a in alu:
            d[a]=[]
        for r in records:
            subfamily = r.id.split("_")[2]
            if subfamily in d.keys():
                d[subfamily].append(r)
        mkdir("data/" + g)
        for sub, seqs in d.items():
            path = "data/" + g + "/" + g + "_" + sub + ".p"
            pickle.dump(seqs, open(path, "wb"))
        
        
def get_frequencies(species):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    freq = {}
    for r in records:
        subfamily = r.id.split("_")[2]
        freq[subfamily] = freq.get(subfamily, 0) + 1
    #uncomment these next three lines out if you want the relative frequencies
#     factor=1.0/sum(freq.values())
#     for sub in freq:
#         freq[sub] = freq[sub]*factor
    return freq

#return a list of sequences for a given species and subfamily
#ex: get_subfamily("humans", "AluJo")
def get_subfamily(species, subfamily):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if subfamily in record.id]

##return a list of sequences for a given species and subfamily on a chromosome
#ex: get_subfamily_chr("humans", "AluJo", 1)
def get_subfamily_chr(species, subfamily, chromosome):
    records = get_subfamily(species, subfamily)
    return [record for record in records if "chr"+str(chromosome) in record.description]

#retrieve a sequence for a given speicies based on its description
def get_sequence(species, description):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if description == record.description]

#parses location information from description string
def get_location(description):
    location = description.split(' ')[1].split(':')
    start, end = location[1].split('-')
    return [location[0].split("=")[1], int(start), int(end)]

# #reverse lookup for sequence description given species, single chromosome, start, and end 
# def description_lookup(species, subfamily, chromosome, start, end): 
#     sequences = pickle.load(file=open("data/"+ species + "/" + species +"_" + subfamily + ".p", "rb"))
#     description = chromosome+":"+str(start) + "-" + str(end) 
#     for s in sequences: 
#         desc = s.description.split(" ")[1][6:]
#         if desc == description: 
#             return s.seq 

#reverse lookup for sequence given lists of chromosomes, start, and end
def description_lookup(species, subfamily, chromosomes, starts, ends): 
    assert len(chromosomes) == len(starts) and len(starts) == len(ends) and len(chromosomes) == len(ends)
    lookup = {} 
    result = []
    sequences = pickle.load(file=open("data/"+ species + "/" + species +"_" + subfamily + ".p", "rb"))
    for s in sequences: 
        desc = s.description.split(" ")[1][6:]
        lookup[desc] = s.seq
    descriptions = [str(chromosomes[i]) + ":" + str(starts[i]) + "-" + str(ends[i]) for i in range(len(chromosomes))] 
    for d in descriptions: 
        result.append(lookup[d])
    return result         

In [25]:
get_frequencies("humans")

{'Alu': 4658,
 'AluJb': 131759,
 'AluJo': 81375,
 'AluJr': 88503,
 'AluJr4': 20966,
 'AluSc': 36338,
 'AluSc5': 7018,
 'AluSc8': 23028,
 'AluSg': 38681,
 'AluSg4': 7603,
 'AluSg7': 9159,
 'AluSp': 53809,
 'AluSq': 19866,
 'AluSq10': 2165,
 'AluSq2': 63875,
 'AluSq4': 1906,
 'AluSx': 123022,
 'AluSx1': 123492,
 'AluSx3': 34198,
 'AluSx4': 11540,
 'AluSz': 107707,
 'AluSz6': 49944,
 'AluY': 110881,
 'AluYa8': 368,
 'AluYd8': 241,
 'AluYe5': 1378,
 'AluYf1': 2025,
 'AluYg6': 899,
 'AluYh9': 165,
 'AluYi6': 470,
 'AluYk11': 1341,
 'AluYk12': 219}

# Single Threaded Code

In [78]:
def generate_pairings(species1, species2, subfamily):
    species1_records = get_subfamily(species1, subfamily)
    species2_records = get_subfamily(species2, subfamily)
    filename = species1 + "_" + species1 + "_" + subfamily + ".csv"
    file=open(filename,"w+")
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    f = IntProgress(min=0, max=len(species1_records))
    display(f)
    for sequence1 in species1_records:
        matches = [distance(str(sequence1.seq), str(sequence2.seq)) for sequence2 in species2_records]
        match = species2_records[np.argmin(matches)]
        location1 = get_location(sequence1.description)
        location2 = get_location(match.description)
        data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence1.seq)], [str(match.seq)]]) 
        wr.writerow(data)
        f.value += 1
    file.close()

In [ ]:
generate_pairings("humans", "chimps", "AluJo")

# Multithreaded Code

In [ ]:
# set parameters here
species1 = "humans"
species2 = "chimps"
subfamily = "AluJo"

In [ ]:
# then run this cell
print("loading " + species1 + " data...")
species1_records = get_subfamily(species1, subfamily)
print("loading " + species2 + " data...")
species2_records = get_subfamily(species2, subfamily)
generate_pairings()

In [ ]:
def find_match(sequence):
    matches = [distance(str(sequence.seq), str(sequence2.seq)) for sequence2 in species2_records]
    match = species2_records[np.argmin(matches)]
    location1 = get_location(sequence.description)
    location2 = get_location(match.description)
    data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence.seq)], [str(match.seq)]]) 
    return data
    
def generate_pairings():
    filename = species1 + "_" + species2 + "_" + subfamily + ".csv"
    print("starting matching")
    p = mp.Pool(mp.cpu_count() + 2)
    with open(filename, 'w+') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        with tqdm(total=len(species1_records), unit="match") as pbar:
            for i, result in tqdm(enumerate(p.imap(func=find_match, iterable=species1_records))):
                # (filename, count) tuples from worker
                pbar.update()
                wr.writerow(result)

In [6]:
h = SeqIO.parse("data/" + "humans" + ".fasta", "fasta")

In [7]:
c = SeqIO.parse("data/" + "chimps" + ".fasta", "fasta")

In [8]:
distance(str(next(h).seq),str(next(c).seq))

63

In [79]:
a = iter([1, 2, 3, 4, 5, 6, 7, 8])

In [84]:
next(a)

2

In [11]:
genomes = [f for f in listdir("data") if "_" not in f]
genomes.remove("Humans")
tasks = [pickle.load(file=open("tasks/" + f, "rb")) for f in listdir("tasks")]
tasks = [t for t in tasks if t.species1=="Humans"]

In [98]:
for a in alu[:1]:
    strings = []
    row=np.inf
    for g in genomes:
        t = [t for t in tasks if t.species2==g and t.subfamily==a][0]
        row_count = sum(1 for row in csv.reader(open("results/"+t.filename()+".csv", "r")))
        row = min(row, row_count)
    row=20
    t = [t for t in tasks if t.subfamily==a][0]
    df = pd.read_csv("results/"+t.filename()+".csv", header=None)
    strings.append(df[df.columns[8]][:row].str.cat())
    for g in genomes:
        t = [t for t in tasks if t.species2==g and t.subfamily==a][0]
        df = pd.read_csv("results/"+t.filename()+".csv", header=None)
        strings.append(df[df.columns[9]][:row].str.cat())
    print(len(strings))
    ls = [len(s) for s in strings]
    print(ls)
    minl = min([len(s) for s in strings])
    with open("clustal/"+a+"smallish.fasta", "w+", newline="") as file:
        wr = csv.writer(file)
        wr.writerow([">Humans_"+a])
        wr.writerow([strings[0]])
        for i in range(1, len(strings)):
            wr.writerow([">"+genomes[i-1]+"_"+a])
            wr.writerow([strings[i]])

13
[5758, 5725, 5732, 5662, 5710, 5768, 5767, 5750, 5692, 5734, 5904, 5666, 5699]


In [17]:
%timeit run_task(tasks[0])

31.1 s ± 93.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def get_location(description):
    location = description.split(' ')[1].split(':')
    start, end = location[1].split('-')
    return [location[0].split("=")[1], int(start), int(end)]

def run_task(task):
    batch_size = min(20, task.remaining())
    # print(data)
    species1_records = pickle.load(open("data/" + task.species1 + "/" + task.species1 + "_" + task.subfamily + ".p", "rb"))[task.completed:task.completed+batch_size]
    species2_records = pickle.load(open("data/" + task.species2 + "/" + task.species2 + "_" + task.subfamily + ".p", "rb"))
    datas = []
    for sequence1 in species1_records:
        # matches = [editdistance.eval(str(sequence1.seq), str(sequence2.seq)) for sequence2 in species2_records]
        matches = [edlib.align(str(sequence1.seq), str(sequence2.seq), task = "editDistance")["editDistance"] for sequence2 in species2_records]
        min_index = np.argmin(matches)
        match = species2_records[min_index]
        location1 = get_location(sequence1.description)
        location2 = get_location(match.description)
        data = np.concatenate([location1, location2, [matches[min_index]], [abs(location1[1] - location2[1])], [str(sequence1.seq)], [str(match.seq)]])
        datas.append(data)
    return task

KeyboardInterrupt: 

In [19]:
df = pd.read_csv("results/"+tasks[0].filename()+".csv", header=None)

In [47]:
df[df.columns[8]][np.random.permutation(len(df)-50)][:10].str.cat()

'GCCAGGCACAGTGGCTCATGCCTATAATCCCAGCTCTTTGGGAGGTCAAGGCGGGCAGATTGCTTGAGCCCAGGAGTTCAAGACTAGCCTGGGCAACATAGTGAGATCTGTCACTACTAAAAATAAAAAAGTTAGTTGGGCGTAGTGGCATGCACCTAGTATTCCCAGCTACTAAGGAGGCTAAGGCAGGAGGATTACTTGAGTCCGGGAGGTTGAGGTTGCAGTGAGCTGATCATGCCATACTACTCCAGCCTGGGCTACAGAGCGAGACCCTGTCTTGAAGAAAAAAAAAAAGGCCAGACACAATGGCTCACACCTGTAATCTCAGCACTTTGGGAGACTGAAGTGGGCAGATCACTTGAGCCCAGGAGTTTGAGACCAGCCTGGGTAATATAGTGAGACCACCGTCTCTACAAAAATACAAAAATTAGCTGGGCATGGTGGTGCACCTGTGGTCCCAGCTACTCGAGAGGCTGAAGTGGGAGGATCACCTGAGCCCAGGAGGTTAAGGCTGCAGTGAGCCGAGATCACGCACCGTACTCCAGCCTGGGTGACAGAGTGAGACCATGTCTCAAAAAAAAAAAAGGCTAGGCACAGTGGCTCACACCTGTAAGCCCAACACTTTGGGAGGCTGAGGTAGGAAGATTGAGTTCAGGAATTCAAGACTAACCTGGGCAACATAGTGAGACCTCATCTCTACTAAAAATTTAAGAAATTAGCTGGGCGTGGTGGCATGTGCCTATAGTCCCAGCTGCAGTCCCAGCTACTCAGGAGGCTGAGGTGAAAGGACGGCTTGAGCCTGGGAGGTTAAGGCTGCAGTAGTCTTGATCACACCACTGCACTACAAGTCCGGGTGACAGAGTGAAACCCTGCCTCCAATAAAAAAAAAAAGAAAGAAAGAAAAAATTAGCCGGGCATGGTGGCACCTGCCTGTGATCCCAGCTACTTGGGAGTTTACTTGGGCCTGGGAGGTAGAAGCTGCACTGAGCCGTGATGGTG

In [38]:
np.random.permutation(len(df))

array([  2, 217, 557, 516,  24, 162,  26, 174, 477, 404, 178, 273, 554,
         1,  42,  43, 372, 242, 134,  27, 575, 193,  58, 194, 327, 186,
       547,  25,  81, 496, 172, 587, 306, 449, 116, 227, 337, 167, 565,
       100, 150, 214,  83, 454,   7, 431, 148, 245, 544, 102,  38, 323,
       358, 405,  33, 467, 313,  97,  49, 301, 156, 187, 275, 324, 266,
       308, 435, 179, 383, 182, 231, 529,  50,  55, 410, 486, 257,  87,
       535, 283,   9, 513, 122,  93, 142, 244, 334, 265,  32,  19, 480,
       338,  72,  48, 318, 200, 281, 497, 274, 183, 586, 287, 310, 413,
       491,  90, 104, 169, 425, 246, 508,  59, 195,  73, 341, 584, 149,
       199,  35, 230, 270, 391,  65, 123, 492, 125, 514, 533, 127, 384,
       299, 555,  86, 571,   8, 415, 568, 368, 233, 263, 559, 466, 262,
       362,  54, 447, 158, 403, 519, 421, 379, 234, 294,  21, 475, 208,
       550, 417,   4, 225, 295, 105, 332, 414, 463, 309, 128, 385,  95,
        88,  37, 111, 576, 345, 373, 328, 307,   5, 185, 259, 30

In [46]:
a = [1, 2, 3, 4]
b = []
[b.append(c) for c in a if c==1]
b

[1]